# Workflow IT_Equipment 

Workflow qui permet l'interaction entre la base 6SIGMA (mis à jour depuis les fichier IT_Equipments), la CMDB (asset_new) et du rapport Claude_it (6SIGMA_rapport) afin de permettre aux équipes le suivi des équipements présent dans les salles P1 -> P4

# 6SIGMA (It_equipment) ⋈ CMDB

`IT_Equipment_v2` : Table 6SIGMA provenant du fichier IT_equipement et qui se limite aux salles P1 -> P4

In [ ]:
DROP TABLE IF EXISTS `IT_Equipment_v2` ;
CREATE TABLE IT_Equipment_v2
as
SELECT * FROM `IT_Equipment`
where nom_de_la_salle in ('P1', 'P2', 'P3', 'P4')

In [ ]:
UPDATE `IT_Equipment_v2` it
set long_name =''
where long_name is null 

`assets_filtree_v2` : Table assets_new (CMDB) filtrée aux deux localisations : FRVLB0C2 / FRTCRAVA

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_v2` ;
CREATE TABLE assets_filtree_v2
as
select *
from `assets_new`
where (API = 'FRVLB0C2' or API = 'FRTCRAVA') ;

In [ ]:
ALTER TABLE assets_filtree_v2
ADD COLUMN Rack varchar(255),
ADD COLUMN Slot varchar(255),
ADD COLUMN Slot_begin varchar(255),
ADD COLUMN Slot_end varchar(255);

ré-organisation de la table, rajout des positions

In [ ]:
UPDATE assets_filtree_v2
SET 
	Rack = SUBSTRING_INDEX(DALLE, '/', 1)  ,
	Slot = SUBSTRING_INDEX(DALLE, '/', -1) ,
	Slot_begin = SUBSTRING_INDEX(SUBSTRING_INDEX(DALLE, '/', -1), '-', 1) ,
	Slot_end = SUBSTRING_INDEX(DALLE, '-', -1)

`assets_filtree_reduced` : Limitations aux salles P1 -> P4, les equipements en salle 0 sont jugées comme retiré 

`assets_filtree_reduced_not_retired` : Limitations aux salles P1 -> P4

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced` ;
CREATE TABLE assets_filtree_reduced
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4', '0')

In [ ]:
DROP TABLE IF EXISTS `assets_filtree_reduced_not_retired` ;
CREATE TABLE assets_filtree_reduced_not_retired
AS
SELECT ype, subtype, ASSET_ID, NOM_COURT, NOM_LONG, FABRICANT, MODEL, DALLE, SALLE, Rack, Slot_Begin, Slot_End, No_serie, etat, api, tag_partenaire
from `assets_filtree_v2` aa
WHERE SALLE in ('P1', 'P2', 'P3', 'P4')

##### On ajoute la même forme de position que sur 6sigma pour fluidifier les query

In [ ]:

ALTER TABLE `assets_filtree_reduced_not_retired`
ADD COLUMN reperage_id varchar(255) ;

update `assets_filtree_reduced_not_retired`
set reperage_id = concat(Rack,":Slot ", slot_begin) ;

ALTER TABLE `assets_filtree_reduced`
ADD COLUMN reperage_id varchar(255) ;

update `assets_filtree_reduced`
set reperage_id = concat(Rack,":Slot ", slot_begin);

`IT_Equipment_match_position_reduced_v3` : groupement des lignes par positions et affichage d'un seul asset_id par positions + nombres d'equipements sur la même position

In [ ]:
DROP TABLE IF EXISTS IT_Equipment_match_position_reduced_v3_not_retired;
CREATE TABLE IT_Equipment_match_position_reduced_v3_not_retired
as
SELECT it.nom_de_la_salle, it.reperage_id, count(it.reperage_id) as RowNumber,
		concat(aa.asset_id) asset_id

FROM   `IT_Equipment_v2` it  
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.reperage_id = aa.reperage_id
)  
GROUP BY it.reperage_id

## Equipement non présent sur IT_Equipment

#### Equipement 6sigma non présent sur IT_Equipment sur la base de la position (salle, rack, slot de départ)

`IT_Equipment_eq_not_in_cmdb` : Liste des équipements CMDB non présent sur It_Equipment

In [ ]:
DROP TABLE IF EXISTS IT_Equipment_eq_not_in_cmdb ;
CREATE TABLE IT_Equipment_eq_not_in_cmdb
as 
select aa.MODEL, aa.subtype, aa.FABRICANT, aa.NOM_LONG, aa.No_serie, aa.No_serie Serial_number, aa.ASSET_ID,
aa.tag_partenaire asset_tag, aa.tag_partenaire asset_number, aa.SALLE, aa.reperage_id reperage_id_origin, 
aa.reperage_id, NULL end_move_target, null null_1 ,null null_2, null null_3, null null_4, null null_5
from `IT_Equipment_v2` it
RIGHT JOIN `assets_filtree_reduced_not_retired` aa 
ON it.reperage_id = aa.reperage_id
and it.nom_de_la_salle = aa.salle

where it.reperage_id is null and aa.dalle != '' and etat != 'Retired' 
order by reperage_id asc

Beaucoup d'information de 6SIGMA ne sont pas dispo sur la CMDB, on complete donc le nombre de colonne par des valeurs vides (manuellement parceque MariaDB est .....)

In [ ]:
DROP TABLE IF EXISTS IT_Equipment_eq_not_in_cmdb ;
CREATE TABLE IT_Equipment_eq_not_in_cmdb
as 
select 'Equipements IT', '' fill1, '' fill2_, aa.NOM_LONG, aa.tag_partenaire asset_tag, aa.FABRICANT, aa.MODEL,
aa.salle nom_de_la_salle, aa.rack nom_de_baie, aa.slot_begin slot_u, '' fill3, '' fill4,  aa.ype, '' fill5, aa.nom_court, 
'' fill6, '' fill7, '' fill7_2, aa.asset_id,
'' fill8, '' fill9,
'' fill10,'' fill11, '' fill12 ,'' fill12_2 ,'' fill13 ,'' fill14 ,'' fill15 ,'' fill16 ,'' fill17 ,'' fill18 , '' fill19 ,
'' fill20 ,'' fill21 ,'' fill22, '' fill2, '' fill23, '' fill24, '' fill25,'' fill26, '' fill27, '' fill28, '' fill29, 
'' fill30 ,'' fill31 ,'' fill32 ,'' fill33 ,'' fill34 ,'' fill35,'' fill36 ,'' fill37 ,'' fill38 ,'' fill39,
'' fill40, '' fill41, '' fill42, aa.reperage_id,
'' fill43, '' fill44, '' fill45, '' fill46, '' fill47 ,'' fill48 ,'' fill49 ,
'' fill50 ,'' fill51 ,'' fill52 ,'' fill53 ,'' fill54 ,'' fill55 ,''  fill56,'' fill57, '' fill58, '' fill59,
'' fill60,'' fill61
from `IT_Equipment_v2` it
RIGHT JOIN `assets_filtree_reduced_not_retired` aa 
ON it.reperage_id = aa.reperage_id
and it.nom_de_la_salle = aa.salle

where it.reperage_id is null and aa.dalle != '' and etat != 'Retired' 
order by reperage_id asc

## On ajoute les équipements manquants à olivier_it

### ajout d'un attribut `from_sources` qui contient la provenance de l'équipement 

    - 6Sigma
    - CMDB

In [ ]:
alter table IT_Equipment_v2
ADD COLUMN from_source VARCHAR(255) ;

### Ajout des equipements cleaned de la table `IT_equipement_unique_records`

`IT_equipement_unique_records` (de Equipment_move_workflow) permet d'historiser tous les équipements passés dans **6SIGMA**. si un equipement n'est plus présent sur 6sigma, il possède l'attribut clean_date : null et  il y est inséré avec la l'attribut `from_source` = 'CLEANED'

In [ ]:
drop table if EXISTS IT_Equipment_unique_records_cleaned ; 
create table IT_Equipment_unique_records_cleaned
as
select it.* from `IT_Equipment_unique_records` it
LEFT JOIN `IT_Equipment_v2` it2 ON it.ASSET_ID = it2.asset_ID 
WHERE clean_date is not null and it2.asset_id is null

select it.* from `IT_Equipment_unique_records` it
LEFT JOIN `IT_Equipment_v2` it2 ON it.ASSET_ID = it2.asset_ID 
WHERE clean_date is not null and it2.asset_id is null
/*UNION 
select it.* from `IT_Equipment_unique_records` it 
where it.asset_ID = '0' and clean_date is not null
*/

In [ ]:
alter table `IT_Equipment_unique_records_cleaned`
DROP COLUMN SOURCES, 
DROP COLUMN INSERTED, 
DROP COLUMN first_occurence, 
DROP COLUMN last_move_salle,
DROP COLUMN last_move,
DROP COLUMN last_move_date,
DROP COLUMN clean_date

#### ADD ROW : attribut de l'equipement et atrtibut from_source = provenance

In [ ]:
INSERT INTO `IT_Equipment_v2` 
SELECT it.*, "CMDB" FROM `IT_Equipment_eq_not_in_cmdb` it

In [ ]:
update `IT_Equipment_v2`
set from_source = '6SIGMA'
where from_source is null

In [ ]:
insert into `IT_Equipment_v2`
select it.*, 'CLEANED' from `IT_Equipment_unique_records_cleaned` it

In [ ]:
ALTER TABLE IT_Equipment_v2 
ADD COLUMN status_CMDB varchar(255),
ADD COLUMN status_position_CMDB varchar(255)

In [ ]:
ALTER TABLE IT_Equipment_v2 
ADD COLUMN status_asset_id varchar(255),
ADD COLUMN status_position varchar(255),
ADD COLUMN status_name varchar(255),
ADD COLUMN status_etat varchar(255),
ADD COLUMN status_global varchar(255),
ADD COLUMN status_etat_hpe varchar(255),
ADD COLUMN status_spec varchar(255),
ADD COLUMN status_cc varchar(255) ;

## Status d'asset_id et de position

	-  OK : asset id présent sur CMDB
	- KO : Non présent

In [ ]:
UPDATE `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_reduced_not_retired` it2 ON it.ASSET_ID = it2.ASSET_ID 

SET it.status_cmdb = 'OK'

WHERE (it.ASSET_ID != '' or it.ASSET_ID != '' or it2.ASSET_ID != '' or it2.ASSET_ID != '')

In [ ]:
UPDATE `IT_Equipment_v2` it 
SET status_CMDB = 'KO'
WHERE status_CMDB is null

	-  OK :Position présente sur CMDB
	- KO : Non présen

In [ ]:
UPDATE `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_reduced_not_retired`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.reperage_id = aa.reperage_id
)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3_not_retired` it2
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position_CMDB = 'OK'

WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
UPDATE `IT_Equipment_v2` it 
SET status_position_CMDB = 'KO'
WHERE status_position_CMDB is null;

# UPDATE de la table IT_Equipment_v2

### `MATCH ASSET_ID`

attribut `status_asset_id` : 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID" impossibilité de trouver l'asset_id dans la CMDB (assets_filtrée)

In [ ]:
UPDATE `IT_Equipment_v2` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and (it.asset_id != '0' or it.asset_id != '')
SET it.status_asset_id = ( CASE WHEN aa.asset_id is not null then 'Trouvé - bon asset id'
                          ELSE "Non Trouvé - Mauvais Asset ID"
                          END 
                          )


### `MATCH POSITION`

attribut `status_position` : 

    - "Bonne localisation"
    - "KO localisation" : impossibilité de trouver la position dans la CMDB (assets_filtrée)

In [ ]:
UPDATE `IT_Equipment_v2` it 
INNER JOIN `assets_filtree_reduced`aa
ON (
	it.Nom_de_la_salle = aa.Salle
AND
	it.Nom_de_baie = aa.Rack
AND
	it.Slot_U = CONVERT(aa.Slot_Begin, integer)
	)  
    
INNER JOIN `IT_Equipment_match_position_reduced_v3` it2
ON it.reperage_id = it2.reperage_id
and it.nom_de_la_salle = it2.nom_de_la_salle

SET it.status_position = 'Bonne localisation'
WHERE  (
   aa.model not like '%BL460C%'
OR aa.model not like '%BL660C%'
OR aa.model not like '%BL20P%'
OR aa.model not like '%BL860C%'
OR aa.model not like '%BL420C%'
OR aa.model not like '%BL680C%'
OR aa.model not like '%WS460C%'
OR aa.model not like '%BL870C%' )

In [ ]:
UPDATE `IT_Equipment_v2` it 
SET it.status_position = 'KO localisation'
where it.status_position is null

# `Nom`

attribut `status_nom_court` : Priorisation du status affiché 
    
    1. Présence d'asset d'id
        1. "nom_court type OLD"
        2. "nom_long type OLD" 
        3. "OK nom_court"
        4. "OK nom_long"
        5. "Mauvais nom" : nom court/long non présent dans la cmdb
    
    2. Non présence d'asset d'id
        1. "nom_court type OLD"
        2. "nom_long type OLD" 
        3. "OK nom_court"
        4. "OK nom_long"
        5. "Mauvais nom" : nom court/long non présent dans la cmdb

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_court  = it.short_name    
SET it.status_name = "OK nom_court"
where it.short_name != ''

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON aa.nom_long  = it.nom   
SET it.status_name = "OK nom_long"
where it.nom != '' and status_name is null

### `'CMDB-OK-OLD'`

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id
 

SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD')

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name


SET it.status_name = (CASE WHEN nom_court like '%OLD' and  nom !='' THEN "nom_court type OLD"
                      WHEN nom_long like '%OLD' and  short_name !='' THEN "nom_long type OLD"
                      ELSE it.status_name
                      END
                     )

where (aa.nom_court like '%_OLD' or aa.nom_long like '%_OLD' or aa.nom_court like '%-OLD' or aa.nom_long like '%-OLD') and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `IT_Equipment_v2` it
SET it.status_name = "Mauvais nom"
where it.status_name is null

# `Status : Etat`

attribut `status_etat` : Si un équipment est repéré (par son asset_id et/ou sa position ) 

    - "Trouvé - bon asset id"
    - "Non Trouvé - Mauvais Asset ID"
    - "KO Etat" - Aucun ou plusieurs equipments sur la même position, impossible d'établir le lien

In [ ]:
UPDATE `IT_Equipment_v2` it
LEFT JOIN `assets_filtree_reduced` aa
ON aa.asset_id = it.asset_id
and it.asset_id != '0'
SET it.status_etat = aa.etat ;


In [ ]:

UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD" ;

In [ ]:

UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD" ;

In [ ]:

UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_court,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id

SET it.status_etat = aa.etat
where status_name = "nom_court type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `IT_Equipment_v2` it
INNER JOIN `assets_filtree_reduced` aa
ON ( REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.nom 
OR REPLACE(REPLACE(aa.nom_long,'_OLD',''),'-OLD','')  = it.short_name )
AND aa.asset_id = it.asset_id
SET it.status_etat = aa.etat
where status_name = "nom_long type OLD" and status_asset_id != 'Trouvé - bon asset id'

In [ ]:
UPDATE `IT_Equipment_v2` it
 SET it.status_etat = 'KO Etat_CMDB'
where it.status_etat is null

# `Status_spec : Cas spécifique`

attribut `status_spec` : 

    - "Patch Panel"
    - "PCP"

### `Patch Panel`

In [ ]:
UPDATE `IT_Equipment_v2` it 
SET it.Status_spec = "Trouvé - Patch Panel"
where type like '%Patch Panel%' or fabricant in ('Arista', 'Alcatel-Lucent');

### `PCP`

In [ ]:
UPDATE `IT_Equipment_v2` it
SET it.Status_spec = 'Trouvé - PCP Mal documenté'
where (it.Nom like '%compute%' or it.short_name like '%compute%' or reperage_id like '%compute%') or 
(it.Nom like '%_Rack%' or it.short_name like '%_Rack%' or reperage_id like '%_Rack%') and (asset_id is null or asset_id ='0' or asset_id ='');

#### `PCP` : Cas des Spectrum SN2100, ils sont supprimmé du rapport final

In [ ]:
drop table if exists IT_Equipment_SN2100 ;
create table IT_Equipment_SN2100
as
select * from `IT_Equipment_v2`  where modele = 'Spectrum SN2100'
GROUP by nom

In [ ]:
delete from `IT_Equipment_v2` where modele = 'Spectrum SN2100'

In [ ]:
insert into `IT_Equipment_v2`
select it.* from `IT_Equipment_SN2100` it

#### Les Baies I11 -> O11 sont tous référencés comme des PCP, ils sont mal documentés

In [ ]:
update `IT_Equipment_v2` 
SET status_spec = 'PCP : KO'
where nom_de_baie in ('I11', 'J11', 'K11', 'L11', 'M11', 'N11', 'O11')

##### `IT_Equipment_status_eq` : Liste des equipements spéciaux, query à ne pas lancer. Garde un historique

drop table if EXISTS IT_Equipment_status_eq ;
create table IT_Equipment_status_eq
as
select nom, modele, fabricant, asset_id, nom_de_la_salle, reperage_id, status_spec from `IT_Equipment_v2` 
where status_spec is not null ;
alter table IT_Equipment_status_eq
ADD COLUMN key_id INTEGER NOT NULL AUTO_INCREMENT KEY;

#### Insertion de nouveau equipements spéciaux (Patch Panel /PCP) 

In [ ]:
INSERT INTO IT_Equipment_status_eq(nom, modele, fabricant, asset_id, nom_de_la_salle, reperage_id, status_spec)
select it.nom, it.modele, it.fabricant, it.asset_id, it.nom_de_la_salle, it.reperage_id, it.status_spec from `IT_Equipment_v2` it
left join `IT_Equipment_status_eq` it2 ON concat(it.Nom_de_la_salle, it.Reperage_ID) = concat(it2.Nom_de_la_salle, it2.Reperage_ID)
where it2.nom is null and it.status_spec is not null and it.asset_id = '0'

In [ ]:
UPDATE `IT_Equipment_status_eq`
set asset_id = (CASE 
                WHEN lower(status_spec) like '%patch panel%' then concat('patch panel: ', key_id)
                WHEN lower(status_spec) like '%pcp%' then concat('pcp: ', key_id)
                END
                )

In [ ]:
UPDATE `IT_Equipment_v2` it
inner join `IT_Equipment_status_eq` it2 ON concat(it.Nom_de_la_salle, it.Reperage_ID) = concat(it2.Nom_de_la_salle, it2.Reperage_ID)
set it.asset_id = it2.asset_id 
where lower(it.status_spec) like '%pcp%' or lower(it.status_spec) like '%patch panel%'


attribut `status_etat_hpe` : 

    1- pas asset id & pas de localisation
    2- pas asset id & localisation
    3- assed id & localisation => status
    4- asset id & pas localisation ==> "assetid"
    5- PCP ==> "PCP"
    6- Patch Panel => "Patch Panel"

In [ ]:
UPDATE `IT_Equipment_v2` it
SET it.status_etat_hpe = 
    (CASE WHEN status_asset_id != 'Trouvé - bon asset id' AND status_position = 'KO localisation' and status_spec is null THEN 'pas asset id & pas de localisation'
     WHEN status_asset_id != 'Trouvé - bon asset id' AND status_position != 'KO localisation' and status_spec is null THEN 'pas asset id & localisation'
     WHEN status_asset_id = 'Trouvé - bon asset id' AND status_position != 'KO localisation' and status_spec is null THEN status_etat
     WHEN status_asset_id = 'Trouvé - bon asset id' AND status_position = 'KO localisation' and status_spec is null THEN 'asset id & pas de localisation'
     WHEN status_spec is not null then status_spec
     END
    )              
    ;

# `GLOBAL STATUS`

attribut `status_status` : Agrégation de tous les attribus

In [ ]:
UPDATE IT_Equipment_v2 
set status_global = concat(status_asset_id , ' - ' , status_position , ' - ' , status_name, " - " , status_etat );

# `STATUS_CC`

attribut `status_CC` : Aggrégation de tous les attribus

In [ ]:
UPDATE IT_Equipment_v2 
set status_cc = concat(
                       "CHIPRE ", status_etat_hpe ) ;


#### `IT_Equipment_x_olivier_it` : Ajout des commentaires et attribues d'olivier_it sur 6sigma ( Non utilisé)

drop table if EXISTS IT_Equipment_x_olivier_it ;
create table IT_Equipment_x_olivier_it
as
select it.*, it2.modele modele_olivier, it2.type type_olivier, it2.fabricant fabricant_olivier, it2.nom nom_olivier, it2.short_name short_name_olivier, it2.SERIAL_NUMBER SERIAL_NUMBER_olivier, it2.asset_id asset_id_olivier, it2.ASSET_TAG ASSET_TAG_olivier, it2.Asset_Number Asset_Number_olivier, it2.Nom_de_la_salle Nom_de_la_salle_olivier, it2.reperage_id_origin, it2.reperage_id reperage_id_olivier,
it2.End_Move_Target End_Move_Target_olivier, it2.Status_hpe, it2.Salle_Dest, it2.Rack_U_Dest, it2.olivier_index, it2.PROJET_Responsable,
it2.status_CMDB status_CMDB_olivier, it2.status_position_CMDB status_position_CMDB_olivier
from `IT_Equipment_v2` it
LEFT JOIN `olivier_it` it2 ON concat(it.Nom_de_la_salle, it.Reperage_ID) = concat(it2.Nom_de_la_salle, it2.Reperage_ID)
where from_source = '6SIGMA'

## Suppression d'equipement non requis (Provenant de la CMDB)

equipement avec comme slot_u le nom du rack, s'agit surement d'une erreur

SELECT nom, Nom_de_la_salle, Reperage_ID, slot_u, slot_u REGEXP '^[0-9]+$'  from_source FROM `IT_Equipment_v2` WHERE slot_u REGEXP '^[0-9]+$' = 0

In [ ]:
delete FROM `IT_Equipment_v2` WHERE slot_u REGEXP '^[0-9]+$' = 0

# Indicateur

`indicator_IT_Equipment_status` : nombre d'equipement pour un status donnée de l'attribut status_cc

In [ ]:
drop VIEW if EXISTS indicator_IT_Equipment_status ;
create VIEW indicator_IT_Equipment_status
as 
SELECT status_cc, count(*) row_count FROM `IT_Equipment_v2`
GROUP BY status_cc

`indicator_IT_Equipment_add_row_per_salle` : nombre d'equipement par salle et par source (6SIGMA, CLEANED, CMDB)

In [ ]:
drop view if EXISTS indicator_IT_Equipment_add_row_per_salle ;
create view indicator_IT_Equipment_add_row_per_salle
as 
select DISTINCT(from_source), NOM_de_la_salle, count(*) equipment_count from `IT_Equipment_v2`
GROUP BY from_source, NOM_de_la_salle

# 6SIGMA (It_equipment) ⋈ Claude_it

Initialement, le fichier Claude_it était issus de l'analyse faite sur le fichier Olivier_it. Lorsque le pipeline Update_IT_Equipment_report.py est lancé, le fichier Claude_it est inséré au sein de la table Olivier_it_2 qui ne partage plus aucune information en provenance d'olivier_it mais qui en garde seulement la même appelation.

### Ajout de l'attribut KEY_VALUE : attribut permettant de distinguer un equipement à l'aide de : asset_id, salle, position. Rend un equipement unique 

In [ ]:
ALTER TABLE IT_Equipment_v2 
ADD COLUMN key_value varchar(255) ;

In [ ]:
UPDATE IT_Equipment_v2 
set key_value = concat(asset_id, nom_de_la_salle, reperage_id)

In [ ]:
UPDATE olivier_it_2 
set key_value = concat(asset_id, nom_de_la_salle, reperage_id)

## `6SIGMA_report` : rapport it_equipement pour l' equipe 

Le fichier initial, Claude_it est inséré en tant que Olivier_it_2 

L'ajout d'un nouvel attribut ce fait en deux étapes, il faut ajouter à la query finale la colonne supplementaire. AVANT la réinsertion, il faut rajouter la colonne correspondante au schema SQL qui génère la table (disponible dans le dossier Download\query_folder.

### generation de la table initiale : En commentaire par défaut

CREATE TABLE `olivier_it_2` (
  `modele` varchar(255) DEFAULT NULL,
  `Type` varchar(100) NOT NULL,
  `fabricant` varchar(255) DEFAULT NULL,
  `LONG_NAME` varchar(255) DEFAULT NULL,
  `SHORT_NAME` varchar(100) NOT NULL,
  `SERIAL_NUMBER` varchar(100) NOT NULL,
  `ASSET_ID` varchar(100) NOT NULL,
  `ASSET_TAG` varchar(100) NOT NULL,
  `Asset_Number` varchar(100) NOT NULL,
  `Nom_de_la_salle` varchar(255) DEFAULT NULL,
  `reperage_id_origin` varchar(255) DEFAULT NULL,
  `reperage_id` varchar(100) NOT NULL,
  `End_Move_Target` varchar(100) NOT NULL,
  `Status_hpe`varchar(255) NOT NULL,
  `Salle_Dest` varchar(100) NOT NULL,
  `Rack_U_Dest` varchar(100) NOT NULL,
  `olivier_index` varchar(100) NOT NULL,
  `PROJET_Responsable` varchar(100) NOT NULL,
  `A_FAIRE_VALIDER` varchar(100) NOT NULL,
  `Valide_O_N` varchar(100) NOT NULL,
  `AUTRE_PROPOSITION_ou_CONFIRMER` varchar(100) NOT NULL,
  `Termine_Oui_Non` varchar(100) NOT NULL,
  `Index_CC` varchar(100) NOT NULL,
  `6SigmaC2` varchar(100) NOT NULL,
  `Chipre_installe` varchar(100) NOT NULL,
  `status_cc` varchar(100) NOT NULL,
  `column1` varchar(100) NOT NULL,
  `status_asset_id` varchar(100) NOT NULL,
  `status_position` varchar(100) NOT NULL,
  `status_name` varchar(100) NOT NULL,
  `status_etat` varchar(100) NOT NULL,
  `CMDB_Servers_Status` varchar(100) NOT NULL,
  `CMDB_Servers_Owner` varchar(100) NOT NULL,
  `key_value` varchar(100) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

drop TABLE if exists 6SIGMA_;
CREATE TABLE 6SIGMA_
as
SELECT
it.Modele, it.Type, it.Fabricant, it.LONG_NAME, it.SHORT_NAME, it.SERIAL_NUMBER, it.ASSET_ID, it.ASSET_TAG, it.ASSET_TAG asset_number, it.Nom_de_la_salle, it2.Reperage_ID_origin, it.Reperage_ID,
it2.End_Move_Target, it2.STATUS_hpe , it2.salle_Dest, 
it2.Rack_U_Dest, it2.olivier_index, it2.PROJET_Responsable, it2.A_FAIRE_VALIDER, it2.Valide_O_N, it2.AUTRE_PROPOSITION_ou_CONFIRMER, it2.Termine_Oui_Non, it2.Index_CC,
it.from_source, it.status_CMDB,it.status_position_cmdb, it.status_asset_id, it.status_position, it.status_name, it.status_etat, it.status_global, it.status_etat_hpe, it.status_spec, it.status_cc status_chipre, null status_cc, null status_cc_2, null status_cc_3, it2.CMDB_Servers_Status, it2.CMDB_Servers_Owner, it.Hauteur_U, it.key_value
FROM `IT_Equipment_v2` it
LEFT JOIN `olivier_it_2` it2 ON it.key_value = concat(it2.asset_id, it2.nom_de_la_salle, it2.reperage_id)
where it.asset_id is not null

### Génération du rapport final

#### Schema table finale

CREATE TABLE `olivier_it_2` (
  `modele` varchar(255) DEFAULT NULL,
  `Type` varchar(100) NOT NULL,
  `fabricant` varchar(255) DEFAULT NULL,
  `nom` varchar(255) DEFAULT NULL,
  `LONG_NAME` varchar(255) DEFAULT NULL,
  `SHORT_NAME` varchar(100) NOT NULL,
  `SERIAL_NUMBER` varchar(100) NOT NULL,
  `ASSET_ID` varchar(100) NOT NULL,
  `ASSET_TAG` varchar(100) NOT NULL,
  `Asset_Number` varchar(100) NOT NULL,
  `Nom_de_la_salle` varchar(255) DEFAULT NULL,
  `reperage_id_origin` varchar(255) DEFAULT NULL,
  `reperage_id` varchar(100) NOT NULL,
  `End_Move_Target` varchar(100) NOT NULL,
  `Status_hpe`varchar(255) NOT NULL,
  `Salle_Dest` varchar(100) NOT NULL,
  `Rack_U_Dest` varchar(100) NOT NULL,
  `olivier_index` varchar(100) NOT NULL,
  `PROJET_Responsable` varchar(100) NOT NULL,
  `A_FAIRE_VALIDER` varchar(100) NOT NULL,
  `Valide_O_N` varchar(100) NOT NULL,
  `AUTRE_PROPOSITION_ou_CONFIRMER` varchar(100) NOT NULL,
  `Termine_Oui_Non` varchar(100) NOT NULL,
  `Index_CC` varchar(100) NOT NULL,
  `from_source` varchar(100) NOT NULL,
  `status_CMDB` varchar(100) NOT NULL,
  `status_position_cmdb` varchar(100) NOT NULL,
  `status_asset_id` varchar(100) NOT NULL,
  `status_position` varchar(100) NOT NULL,
  `status_name` varchar(100) NOT NULL,
  `status_etat` varchar(100) NOT NULL,
  `status_global` varchar(100) NOT NULL,
  `status_etat_hpe` varchar(100) NOT NULL,
  `status_spec` varchar(100) NOT NULL,
  `status_cc` varchar(100) NOT NULL,
  `status_cc_2` varchar(100) NOT NULL,
  `status_cc_3` varchar(100) NOT NULL,
  `CMDB_Servers_Status` varchar(100) NOT NULL,
  `CMDB_Servers_Owner` varchar(100) NOT NULL,
  `key_value` varchar(100) NOT NULL
  `date_apparition` varchar(100) NOT NULL,
  `date_dernier_mouvement` varchar(100) NOT NULL,
  `date_clean` varchar(100) NOT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8;

###

In [ ]:
drop TABLE if exists 6SIGMA_;
CREATE TABLE 6SIGMA_
AS
SELECT
it.Modele, it.Type, it.Fabricant, it.nom, it.LONG_NAME, it.SHORT_NAME, it.SERIAL_NUMBER, it.ASSET_ID, it.ASSET_TAG, it.ASSET_TAG asset_number, it.Nom_de_la_salle, it2.Reperage_ID_origin, it.Reperage_ID,
it2.End_Move_Target, it2.STATUS_hpe , it2.salle_Dest, 
it2.Rack_U_Dest, it2.olivier_index, it2.PROJET_Responsable, it2.A_FAIRE_VALIDER, it2.Valide_O_N, it2.AUTRE_PROPOSITION_ou_CONFIRMER, it2.Termine_Oui_Non, it2.Index_CC,
it.from_source, it.status_CMDB,it.status_position_cmdb, it.status_asset_id, it.status_position, it.status_name, it.status_etat, it.status_global, it.status_etat_hpe, it.status_spec, it2.status_cc status_cc, it2.status_cc_2 status_cc_2, it2.status_cc_3 status_cc_3, it2.CMDB_Servers_Status, it2.CMDB_Servers_Owner, it.Hauteur_U , it.key_value
FROM `IT_Equipment_v2` it
LEFT JOIN `olivier_it_2` it2 ON concat(it.key_value, it.from_source ) = concat(it2.key_value, it2.from_source)
where it.asset_id is not null and it.from_source != 'CMDB'

In [ ]:
drop TABLE if exists CMDB_;
CREATE TABLE CMDB_
AS
SELECT
it.Modele, it.Type, it.Fabricant, it.nom, it.LONG_NAME, it.SHORT_NAME, it.SERIAL_NUMBER, it.ASSET_ID, it.ASSET_TAG, it.ASSET_TAG asset_number, it.Nom_de_la_salle, it2.Reperage_ID_origin, it.Reperage_ID,
it2.End_Move_Target, it2.STATUS_hpe , it2.salle_Dest, 
it2.Rack_U_Dest, it2.olivier_index, it2.PROJET_Responsable, it2.A_FAIRE_VALIDER, it2.Valide_O_N, it2.AUTRE_PROPOSITION_ou_CONFIRMER, it2.Termine_Oui_Non, it2.Index_CC,
it.from_source, it.status_CMDB,it.status_position_cmdb, it.status_asset_id, it.status_position, it.status_name, it.status_etat, it.status_global, it.status_etat_hpe, it.status_spec, it2.status_cc_2 status_cc_2, it2.status_cc_3 status_cc_3, it2.CMDB_Servers_Status, it2.CMDB_Servers_Owner, it.Hauteur_U , it.key_value
FROM `IT_Equipment_v2` it
LEFT JOIN `olivier_it_2` it2 ON concat(it.key_value, it.from_source ) = concat(it2.key_value, it2.from_source)
where it.asset_id is not null and it.from_source = 'CMDB'

### `IT_Equipment_unique_records_move` : ajout des dates d'apparition, des dates du dernier mouvement et des dates de clean des equipements au rapport 6SIGMA 

In [ ]:
DROP TABLE IF EXISTS IT_Equipment_unique_records_move ;
CREATE TABLE IT_Equipment_unique_records_move
as 
SELECT concat(nom, ASSET_ID) key_value, nom, long_name, ASSET_ID, min(first_occurence) date_apparition, max(last_move_date) date_dernier_mouvement, GROUP_CONCAT(IFNULL(clean_date, 'null')) listing_date_clean, if( GROUP_CONCAT(IFNULL(clean_date, 'null')) not like '%null%', clean_date, NULL) date_clean  
FROM `IT_Equipment_unique_records` 
group by concat(nom, ASSET_ID)

### `6SIGMA_report` : Table final du rapport 6SIGMA, l'ajout d'un nouvel attribut est à effectuer sur cette query

La clé unique n'est plus la combinaison asset_id et position mais asset_id et nom. Si l'equipement bouge on veut pouvoir récuperer l'information pour un nom et son asset_id.

## Attention, afin de faciliter l'extraction des données vers EXCEL, les dates étaient au format ANNEE-MOIS-JOUR, lors de la création du csv 6SIGMA, le format des dates est inversés pour devenir JOUR-MOIS-ANNEE. Cependant, le format initial reste valide dans toutes les autres tables, seul les tables 6SIGMA_report/CMDB_report sont impactées 

In [ ]:
drop TABLE if exists 6SIGMA_report;
CREATE TABLE 6SIGMA_report
select it.*, 
date_format(str_to_date(it2.date_apparition, '%y-%m-%d'), '%d-%m-%y') date_apparition, 
date_format(str_to_date(it2.date_dernier_mouvement, '%y-%m-%d'), '%d-%m-%y') date_dernier_mouvement,
date_format(str_to_date(it2.date_clean, '%y-%m-%d'), '%d-%m-%y') date_clean from `6SIGMA_` it
LEFT JOIN `IT_Equipment_unique_records_move` it2 ON concat(it.nom, it.ASSET_ID) = concat(it2.nom, it2.ASSET_ID)
where it.asset_id is not null

In [ ]:
drop TABLE if exists CMDB_report;
CREATE TABLE CMDB_report
select it.*,
date_format(str_to_date(it2.date_apparition, '%y-%m-%d'), '%d-%m-%y') date_apparition, 
date_format(str_to_date(it2.date_dernier_mouvement, '%y-%m-%d'), '%d-%m-%y') date_dernier_mouvement,
date_format(str_to_date(it2.date_clean, '%y-%m-%d'), '%d-%m-%y') date_clean from `CMDB_` it
LEFT JOIN `IT_Equipment_unique_records_move` it2 ON concat(it.nom, it.ASSET_ID) = concat(it2.nom, it2.ASSET_ID)
where it.asset_id is not null

#### `CMDB_TO_FILTER` : filtre en dur permettant de supprimer certain modele d'equipement sur la CMDB

In [ ]:
delete it from `CMDB_report` as it
INNER join `CMDB_TO_FILTER` fi on it.MODELE = fi.MODELE

### Control /check

drop table if EXISTS to_check ;
create table to_check
as
select  concat(it.asset_id, it.nom_de_la_salle, it.reperage_id ) asset_salle_reperage , count(*) row_number from `IT_Equipment_v2` it 
group by  concat(it.asset_id, it.nom_de_la_salle, it.reperage_id )
HAVING row_number > 1
order by count(*) desc

drop table if EXISTS to_check_csv ;
create table to_check_csv 
as
select * from `IT_Equipment_v2` it 
INNER JOIN `to_check` it2 ON concat(it.asset_id, it.nom_de_la_salle, it.reperage_id ) = it2.asset_salle_reperage
ORDER by nom
